In [11]:
import pandas as pd
import numpy as np

### Загрузка данных

In [12]:
raw_data = pd.read_csv("data/raw/NASDAQ_FULL_2021_Close.csv")
raw_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/NASDAQ_FULL_2021_Close.csv'

In [ ]:
raw_data.drop(columns=["Date"], inplace=True)
raw_data.head()

### Фильтрация активов

In [ ]:
raw_data.isna().sum(axis=0).value_counts().sort_index()

Удалим те, что имеют более 50 пропусков (n/a). Это те активы, по которым либо не нашлось данных (ещё не обращались на бирже), либо листинг которых произошёл в 2021 году (например, в его середине), и полных данных по ним нет.

In [ ]:
MAX_NA_THRESHOLD = 50

In [ ]:
na_counts = raw_data.isna().sum(axis=0)
valid_tickers = na_counts[na_counts <= MAX_NA_THRESHOLD].index

data = raw_data[valid_tickers]

In [ ]:
print(f"Tickers was: {data.shape[1]}")
print(f"Deleted tickers (NA > {MAX_NA_THRESHOLD}): {raw_data.shape[1] - data.shape[1]}")
print(f"Tickers left for analysis: {data.shape[1]}")

### Заполнение пропусков

In [ ]:
data.ffill(inplace=True)
data.bfill(inplace=True)

Проверим, что мы точно почистили все n/a:

In [ ]:
data.isna().sum(axis=0).value_counts()

Проверим, нет ли отрицательных цен в данных

In [ ]:
# Удаление активов с аномальными ценами (Отрицательные или Нулевые)
# Цены акций не могут быть отрицательными. Если после Imputation появились отрицательные
# или нулевые значения, это серьезная ошибка данных. Удаляем такие тикеры.

has_non_positive_price = (data <= 0).any(axis=0)
non_positive_tickers = has_non_positive_price[has_non_positive_price].index.tolist()

print(f"Total tickers with non-positive price found: {len(non_positive_tickers)}")
print(f"Examples of non-positive tickers: {non_positive_tickers[:10]} ...")

initial_columns = data.shape[1]
# Drop columns (tickers) that have at least one non-positive price
data = data.drop(columns=non_positive_tickers, errors='ignore')
deleted_count = initial_columns - data.shape[1]

print(f"Removed {deleted_count} assets with non-positive prices.")
print(f"Tickers left after price cleaning: {data.shape[1]}")

# Финальная проверка чистоты данных
if (data <= 0).any().any():
    print("Error: Non-positive prices remain in data!")
elif data.isna().sum().sum() > 0:
    print("Error: N/A values remain in data!")
else:
    print("Data is clean and ready for return calculation.")

Экспортируем очищенные данные:

In [ ]:
data.to_csv("data/processed/NASDAQ_FULL_2021_Cleaned_Imputed.csv", index=False)

# Nasdaq 100

In [18]:
raw_data = pd.read_csv("data/raw/NSDQ100-Price (daily)-2021-2021.txt", sep="\t")
raw_data.head()

,ticker,AAPL,ABNB,ADBE,ADI,ADP,ADSK,AEP,AMAT,AMD,...,VRTX,WBA,WBD,WDAY,XEL,ZS,AU10Y,BR10Y,CA10Y,CH10Y
0,2021-01-04,"129,41","139,15","485,34","146,02","168,96","296,84","81,54","86,87","92,3",...,"229,43","41,4","30,63","228,08","65,655","196,09","0,99","6,86","0,677","2,8"
1,2021-01-05,"131,01","148,3","485,69","148,63","168,49","301,7","81,43","89,6","92,77",...,"228,58","41,16","31,52","225,36","65,02","193,48","0,949","6,84","0,713","2,8"
2,2021-01-06,"126,6","142,77","466,31","149,3","167,9","302,87","82,08","90,83","90,33",...,"226,65","43,03","34,25",220,"66,46","184,42","1,061","7,04","0,758","2,81"
3,2021-01-07,"130,92","151,27","477,74","155,61","168,85","315,2","79,89","94,56","95,16",...,"232,9","45,26","34,4","226,21","65,16","193,05","1,058","7,27","0,794","2,83"
4,2021-01-08,"132,05","149,77","485,1","156,74","171,05","319,85","79,18","95,56","94,58",...,237,"45,21","34,63","227,65","65,76","197,37","1,096","7,57","0,815","2,85"


In [19]:
raw_data.columns

Index(['ticker', 'AAPL', 'ABNB', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'AMAT',
       'AMD',
       ...
       'VRTX', 'WBA', 'WBD', 'WDAY', 'XEL', 'ZS', 'AU10Y', 'BR10Y', 'CA10Y',
       'CH10Y'],
      dtype='object', length=101)

In [20]:
raw_data.drop(columns=["ticker"], inplace=True)

In [21]:
raw_data.isna().sum(axis=0).value_counts().sort_index()

,count
0,96
1,1
5,1
6,2


In [23]:
raw_data.ffill(inplace=True)
raw_data.bfill(inplace=True)

In [24]:
raw_data.isna().sum(axis=0).value_counts().sort_index()

,count
0,100


In [25]:
raw_data.to_csv("data/processed/NASDAQ_100_2021_Cleaned.csv", index=False)